In [172]:
import pandas as pd
import numpy as np
from scipy.optimize import root_scalar,fsolve

In [173]:
df = pd.read_excel("RBAbondyields.xlsx")[10:].reset_index(drop=True)
df.columns = ["Date","24", "36", "60", "120"]
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
df[["24", "36", "60", "120"]] = df[["24", "36", "60", "120"]].apply(np.float32)
df["0"] = 0
df 


,Date,24,36,60,120,0
0,2013-09-02,2.594,2.800,3.258,3.989,0
1,2013-09-03,2.652,2.857,3.312,4.030,0
2,2013-09-04,2.714,2.925,3.373,4.055,0
3,2013-09-05,2.752,2.968,3.412,4.106,0
4,2013-09-06,2.790,3.007,3.455,4.177,0
...,...,...,...,...,...,...
2656,2024-02-29,3.754,3.702,3.774,4.141,0
2657,2024-03-01,3.764,3.712,3.782,4.146,0
2658,2024-03-04,3.732,3.682,3.752,4.106,0
2659,2024-03-05,3.732,3.682,3.749,4.096,0


$V(t,\mathbb{T},k) = 1 = B(t,T_n) + k\sum_{i=1}^n B(t,T_i)$




In [174]:
def logl_interp(x, x0, x1, y0, y1):
    return np.exp(np.log(y0) + (x - x0) * np.log(y1 / y0) / (x1 - x0))

def compute_term_structure(df):
    results = []
    
    for _, row in df.iterrows():
        def term_24(x):
            return -1 + x + row["24"] / 200 * sum([logl_interp(m, 0, 24, 1, x) for m in range(6, 24, 6)] + [x])
        
        term_struc_24 = root_scalar(term_24, bracket=[0.001, 10]).root
        
        def term_36(x):
            return -1 + x + row["36"] / 200 * sum(
                [logl_interp(m, 0, 24, 1, term_struc_24) for m in range(6, 24, 6)] +
                [term_struc_24] +
                [logl_interp(m, 24, 36, term_struc_24, x) for m in range(30, 36, 6)] +
                [x]
            )
        
        term_struc_36 = root_scalar(term_36, bracket=[0.001, 10]).root
        
        def term_60(x):
            return -1 + x + row["60"] / 200 * sum(
                [logl_interp(m, 0, 24, 1, term_struc_24) for m in range(6, 24, 6)] +
                [term_struc_24] +
                [logl_interp(m, 24, 36, term_struc_24, term_struc_36) for m in range(30, 36, 6)] +
                [term_struc_36] +
                [logl_interp(m, 36, 60, term_struc_36, x) for m in range(42, 60, 6)] +
                [x]
            )
        
        term_struc_60 = root_scalar(term_60, bracket=[0.001, 10]).root
        
        def term_120(x):
            return -1 + x + row["120"] / 200  * sum(
                [logl_interp(m, 0, 24, 1, term_struc_24) for m in range(6, 24, 6)] +
                [term_struc_24] +
                [logl_interp(m, 24, 36, term_struc_24, term_struc_36) for m in range(30, 36, 6)] +
                [term_struc_36] +
                [logl_interp(m, 36, 60, term_struc_36, term_struc_60) for m in range(42, 60, 6)] +
                [term_struc_60] +
                [logl_interp(m, 60, 120, term_struc_60, x) for m in range(66, 120, 6)] +
                [x]
            )
        
        term_struc_120 = root_scalar(term_120, bracket=[0.001, 10]).root
        
        results.append([row["Date"], term_struc_24, term_struc_36, term_struc_60, term_struc_120])
    
    return pd.DataFrame(results, columns=["Date", "TS_24", "TS_36", "TS_60", "TS_120"])


term_structure_df = compute_term_structure(df)
print(term_structure_df)

           Date     TS_24     TS_36     TS_60    TS_120
0    2013-09-02  0.949760  0.919807  0.849641  0.667184
1    2013-09-03  0.948673  0.918255  0.847375  0.664542
2    2013-09-04  0.947513  0.916402  0.844826  0.663155
3    2013-09-05  0.946802  0.915231  0.843198  0.659680
4    2013-09-06  0.946093  0.914173  0.841393  0.654766
...         ...       ...       ...       ...       ...
2656 2024-02-29  0.928315  0.895846  0.829368  0.660893
2657 2024-03-01  0.928133  0.895582  0.829047  0.660592
2658 2024-03-04  0.928716  0.896372  0.830268  0.663289
2659 2024-03-05  0.928716  0.896372  0.830397  0.664004
2660 2024-03-06  0.929738  0.897906  0.833115  0.669703

[2661 rows x 5 columns]


In [175]:
term_structure_df["TS_0"] = 1
def interpolate_term_structure(term_structure_df,maturity):
    
    if maturity in [0,24,36,60,120]:
        return term_structure_df["TS_"+str(maturity)]
    x0 = max([_ for _ in [0, 24, 36, 60, 120] if _ < maturity])
    x1 = min([_ for _ in [0, 24, 36, 60, 120] if _ > maturity])

    return logl_interp(maturity,x0,x1,term_structure_df["TS_"+str(x0)],term_structure_df["TS_"+str(x1)])


In [176]:
term_structure_df[term_structure_df["Date"]=="2023-04-21"]

,Date,TS_24,TS_36,TS_60,TS_120,TS_0
2438,2023-04-21,0.94159,0.911119,0.85374,0.707457,1


In [177]:
def compute_bond(term_structure_df,T,r,is_duration=False,is_convexity=False):
    B = 0
    start = T%6 if T%6!=0 else 6
    for maturity in range(start,T+6+T%6,6):
        b = interpolate_term_structure(term_structure_df,maturity)
        if is_duration:
            B+= b*maturity/12
        elif is_convexity:
            B+= b*(maturity/12)**2

        else:
            B+= b

    b = interpolate_term_structure(term_structure_df,T)
    if is_duration:
        B = B*r/200 + b*T/12
    elif is_convexity:
        B = B*r/200 + b*(T/12)**2
    else:
        B = B*r/200 + b 


    return B.values[0]

In [178]:
def duration_FW(term_structure_df,T,r):
    return compute_bond(term_structure_df,T,r,True,False)/compute_bond(term_structure_df,T,r,False,False)

In [179]:
def convexity(term_structure_df,T,r):
    return compute_bond(term_structure_df,T,r,False,True)/compute_bond(term_structure_df,T,r,False,False)#*0.5 +compute_bond(term_structure_df,T,r,True,False)/compute_bond(term_structure_df,T,r,False,False)

In [181]:
B1 = compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],24,3.25)
B2 = compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],36,4.25)
B3 = compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],61,2.25)
B4 = compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],80,1)
print(B1,B2,B3,B4)

1.0041996351772637 1.0320359667106485 0.974798256379853 0.8687655412233806


In [182]:
D1 = duration_FW(term_structure_df[term_structure_df["Date"] == "2023-04-21"],24,3.25)
D2 = duration_FW(term_structure_df[term_structure_df["Date"] == "2023-04-21"],36,4.25)
D3 = duration_FW(term_structure_df[term_structure_df["Date"] == "2023-04-21"],61,2.25)
D4 = duration_FW(term_structure_df[term_structure_df["Date"] == "2023-04-21"],80,1)
print(D1,D2,D3,D4)

1.9526531621733427 2.8508870302463425 4.785611125600002 6.4238024754754965


In [183]:
C1 = convexity(term_structure_df[term_structure_df["Date"] == "2023-04-21"],24,3.25)
C2 = convexity(term_structure_df[term_structure_df["Date"] == "2023-04-21"],36,4.25)
C3 = convexity(term_structure_df[term_structure_df["Date"] == "2023-04-21"],61,2.25)
C4 = convexity(term_structure_df[term_structure_df["Date"] == "2023-04-21"],80,1)
print(C1,C2,C3,C4)

3.866047814058627 8.380449270630212 23.888775610827132 42.32975124129763


We want $\delta_1 V_{B_1}(0,\mathbb{T},k)+ \delta_2 V_{B_2}(0,\mathbb{T},k) = 120M$ and $\delta_1 B_1 D_{FW1}+\delta_2 B_2 D_{FW2} - 120*4 e^{-rT} =0$




In [197]:
def solve_hedge(x,term_structure_df):
    B1 = compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],24,3.25)
    B4 = compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],80,1)
    ZCB = interpolate_term_structure(term_structure_df[term_structure_df["Date"]=="2023-04-21"],48).reset_index(drop=True)[0]
    return [x[0] * B1 + x[1]*B4-120e6*ZCB,x[0]*duration_FW(term_structure_df[term_structure_df["Date"] == "2023-04-21"],24,3.25)*B1+x[1]*duration_FW(term_structure_df[term_structure_df["Date"] == "2023-04-21"],80,1)*B4-4*120e6*ZCB]

delta1,delta2 = fsolve(solve_hedge,[60e6,60e6],term_structure_df)
print(delta1,delta2)

57133320.99555428 55782906.51218997


In [ ]:
delta1 * compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],24,3.25) + delta2*compute_bond(term_structure_df[term_structure_df["Date"] == "2023-04-21"],80,1)

105835527.06727707